In [2]:
from memorymanager import MemManager
from ram import RAM
from arenatest_2 import Arena
from pooltest_2 import Pool
from blocktest_2 import Block

In [3]:

# Step 2: Initialize the memory manager with the RAM instance
ram = RAM()
mem_manager = MemManager(ram)

# Step 3: Add one arena, one pool, and one block
arena = Arena(0)
mem_manager.add_arena(arena)

pool = Pool(0)
mem_manager.add_pool("Arena 1", pool)

block = Block(0)
mem_manager.add_block("Arena 1", "pool 1", block)





In [7]:
mem_manager.memory["arenas"]["arena_obj"].iloc[0].get_remaining_unused_mem()

262144